In [1]:
import os
import openai
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from crewai import Agent, Task, Crew, Process
from IPython.display import Markdown, display

# Set environment variables
os.environ["NVIDIA_API_KEY"] = "Your-NVIDIA-API-key"                   # As an option, an OpenAI model (with the respective OpenAI-API-key) can be used
os.environ["NVIDIA_MODEL_NAME"] = "writer/palmyra-med-70b"
os.environ["NVIDIA_BASE_URL"] = "URL of the model"                     # For an OpenAI model this should not be used

# Initialize the embeddings model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

def create_query_engine(db_path, collection_name):
    db_client = chromadb.PersistentClient(path=db_path)
    chroma_collection = db_client.get_or_create_collection(collection_name)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)
    return index.as_query_engine()

# Define agents

# Agent 1: Sepsis Management Recommendations
sepsis_management_agent = Agent(
    role="Sepsis Management Advisor",
    goal="Provide literature-based recommendations for sepsis management tailored to a patient case.",
    backstory="You are a medical expert specializing in sepsis management, equipped with the latest literature and research findings.",
    verbose=True,
    memory=False,
    custom_code='''
def query_literature_recommendations(vignette):
    query_engine = create_query_engine("The path to the sepsis_management_chroma_db folder", "sepsis_management")
    # Convert the vignette dictionary to a string for querying
    query_str = "\\n".join(f"{key}: {value}" for key, value in vignette.items())
    response = query_engine.query(query_str)
    return str(response)
    ''',
    inputs=['vignette']
)

# Agent 2: Antibiotic Recommendations
antibiotic_recommendation_agent = Agent(
    role="Antibiotic Specialist",
    goal="Provide antibiotic recommendations based on the 'special_comments' field of the patient vignette.",
    backstory="You are an expert in antibiotic therapy, capable of tailoring recommendations to specific patient needs.",
    verbose=True,
    memory=False,
    custom_code='''
def query_antibiotic_recommendations(special_comments):
    query_engine = create_query_engine("The path to the sepsis_antibiotics_chroma_db folder", "sepsis_antibiotic_recommendations")
    # The special_comments is already a string
    response = query_engine.query(special_comments)
    return str(response)
    ''',
    inputs=['special_comments']
)

# Agent 3: Compliance and Detailed Guideline-based Recommendations
guideline_compliance_agent = Agent(
    role="Sepsis Compliance Officer",
    goal="Check if the recommendations comply with current sepsis management guidelines and provide detailed guideline-based case management recommendations.",
    backstory="You are tasked with ensuring that sepsis management adheres to the latest guidelines.",
    verbose=True,
    memory=False,
    custom_code='''
def check_guideline_compliance(literature_recommendations, antibiotic_recommendations, vignette):
    query_engine = create_query_engine("The path to the sepsis_guidelines_chroma_db folder", "sepsis_management_guidelines")
    
    # Combine the recommendations into a single query string
    combined_recommendations = f"Literature Recommendations: {literature_recommendations}\\nAntibiotic Recommendations: {antibiotic_recommendations}"
    compliance_response = query_engine.query(combined_recommendations)
    
    # Accessing the content of the response
    compliance_content = compliance_response.get_content() if hasattr(compliance_response, 'get_content') else str(compliance_response)
    
    if "compliant" in compliance_content:
        compliance_statement = "Recommendations comply with current sepsis management guidelines."
        compliance_explanation = ""
    else:
        compliance_statement = "Recommendations do not comply with current sepsis management guidelines."
        # Query for detailed explanation on why they do not comply
        compliance_explanation_query = f"Explain why the following recommendations do not comply: {combined_recommendations}"
        compliance_explanation_response = query_engine.query(compliance_explanation_query)
        compliance_explanation = str(compliance_explanation_response)

        # Query for suggestions to achieve compliance
        compliance_suggestion_query = f"Provide suggestions to make the following recommendations compliant: {combined_recommendations}"
        compliance_suggestion_response = query_engine.query(compliance_suggestion_query)
        compliance_suggestions = str(compliance_suggestion_response)
        
        compliance_explanation += f"\n\nTo achieve compliance, consider the following suggestions:\n{compliance_suggestions}"
    
    # Convert the vignette dictionary to a string for querying
    vignette_str = "\\n".join(f"{key}: {value}" for key, value in vignette.items())
    
    # Detailed guideline-based recommendations
    # Sepsis Diagnosis Confirmation
    diagnosis_query = f"Provide recommendations for sepsis diagnosis confirmation using SOFA/qSOFA criteria based on the following case: {vignette_str}"
    diagnosis_recommendations = query_engine.query(diagnosis_query)
    
    # Initial Resuscitation and Treatment
    resuscitation_query = f"Provide recommendations for initial resuscitation and treatment, including fluid therapy and source control for the following case: {vignette_str}"
    resuscitation_recommendations = query_engine.query(resuscitation_query)
    
    # Hemodynamic Support and Adjunctive Therapy
    hemodynamic_query = f"Provide recommendations for hemodynamic support and adjunctive therapy, including vasopressors, oxygen, and ventilation support for the following case: {vignette_str}"
    hemodynamic_recommendations = query_engine.query(hemodynamic_query)
    
    # Monitoring and Follow-Up
    monitoring_query = f"Provide recommendations for monitoring and follow-up, including frequent reassessment and laboratory and imaging studies for the following case: {vignette_str}"
    monitoring_recommendations = query_engine.query(monitoring_query)
    
    # Aggregate the detailed guideline-based recommendations
    detailed_guideline_recommendations = f"""
### Sepsis Diagnosis Confirmation
{diagnosis_recommendations}

### Initial Resuscitation and Treatment
{resuscitation_recommendations}

### Hemodynamic Support and Adjunctive Therapy
{hemodynamic_recommendations}

### Monitoring and Follow-Up
{monitoring_recommendations}
    """
    
    return compliance_statement, compliance_explanation, detailed_guideline_recommendations
    ''',
    inputs=['literature_recommendations', 'antibiotic_recommendations', 'vignette']
)

# Define tasks

# Task for Sepsis Management Recommendations    

sepsis_task = Task(
    description="Provide literature-based recommendations for sepsis management for the given patient case.",
    expected_output="Recommendations for sepsis management.",
    agent=sepsis_management_agent,
    async_execution=True,
    inputs={'vignette': 'vignette_input'}
)

# Task for Antibiotic Recommendations
antibiotic_task = Task(
    description="Provide antibiotic recommendations for the given patient case, focusing on the 'special_comments' field.",
    expected_output="Recommendations for antibiotic therapy.",
    agent=antibiotic_recommendation_agent,
    async_execution=True,
    inputs={'special_comments': 'vignette_input["special_comments"]'}
)

# Task for Compliance Check and Guideline-based Recommendations
compliance_task = Task(
    description="Check the recommendations for compliance with current sepsis management guidelines and provide detailed recommendations based on the guidelines.",
    expected_output="Compliance statement and detailed guideline-based recommendations.",
    agent=guideline_compliance_agent,
    async_execution=False,  # This task relies on previous task outputs
    inputs={
        'literature_recommendations': 'sepsis_task.result',
        'antibiotic_recommendations': 'antibiotic_task.result',
        'vignette': 'vignette_input'
    }
)

# Assemble the crew
sepsis_crew = Crew(
    agents=[sepsis_management_agent, antibiotic_recommendation_agent, guideline_compliance_agent],
    tasks=[sepsis_task, antibiotic_task, compliance_task],
    process=Process.sequential  # Use sequential to ensure the third task waits for the first two
)

# Define the patient vignette
vignette_input = {
    'diagnosis': 'Suspected sepsis due to pneumonia',
    'demographic': 'female, 86 years',
    'vital_signs': 'HR 76 bpm, BP 103/49 mmHg, RR 22 rpm, blood O2 saturation 90%, temperature 37.2 degrees Celsius',
    'laboratory_findings': 'WBC 41.100/mm3, blood glucose 225 mg/dL, BUN 13 mg/dL',
    'medical_history': 'Chronic obstructive pulmonary disease, atrial fibrillation, type 2 diabetes mellitus without complications, hypothyroidism',
    'special_comments': 'Pulmonary infection'
}

# Debugging: Manually execute each task step-by-step and print the intermediate results

# Step 1: Get Sepsis Management Recommendations
query_engine_sepsis = create_query_engine("The path to the sepsis_management_chroma_db folder", "sepsis_management")
vignette_str = "\n".join(f"{key}: {value}" for key, value in vignette_input.items())
sepsis_recommendations = query_engine_sepsis.query(vignette_str)
print("Sepsis Management Recommendations:\n", sepsis_recommendations)

# Step 2: Get Antibiotic Recommendations
query_engine_antibiotics = create_query_engine("The path to the sepsis_antibiotics_chroma_db folder", "sepsis_antibiotic_recommendations")
antibiotic_recommendations = query_engine_antibiotics.query(vignette_input['special_comments'])
print("Antibiotic Recommendations:\n", antibiotic_recommendations)

# Step 3: Check Compliance and Get Detailed Guideline Recommendations
query_engine_guidelines = create_query_engine("The path to the sepsis_guidelines_chroma_db folder", "sepsis_management_guidelines")

combined_recommendations = f"Literature Recommendations: {sepsis_recommendations}\nAntibiotic Recommendations: {antibiotic_recommendations}"
compliance_response = query_engine_guidelines.query(combined_recommendations)

# Accessing the content of the response, assuming get_content() or a similar method exists
# You might need to replace get_content() with the actual method to extract text
compliance_content = compliance_response.get_content() if hasattr(compliance_response, 'get_content') else str(compliance_response)

if "compliant" in compliance_content:
    compliance_statement = "Recommendations comply with current sepsis management guidelines."
    compliance_explanation = ""
else:
    compliance_statement = "Recommendations do not comply with current sepsis management guidelines."
    # Detailed explanation and guidance
    compliance_explanation_query = f"Explain why the following recommendations do not comply: {combined_recommendations}"
    compliance_explanation_response = query_engine_guidelines.query(compliance_explanation_query)
    compliance_explanation = str(compliance_explanation_response)
    
    # Suggestions for compliance
    compliance_suggestion_query = f"Provide suggestions to make the following recommendations compliant: {combined_recommendations}"
    compliance_suggestion_response = query_engine_guidelines.query(compliance_suggestion_query)
    compliance_suggestions = str(compliance_suggestion_response)
    
    compliance_explanation += f"\n\nTo achieve compliance, consider the following suggestions:\n{compliance_suggestions}"

# Detailed guideline-based recommendations
diagnosis_query = f"Provide recommendations for sepsis diagnosis confirmation using SOFA/qSOFA criteria based on the following case: {vignette_str}"
diagnosis_recommendations = query_engine_guidelines.query(diagnosis_query)

resuscitation_query = f"Provide recommendations for initial resuscitation and treatment, including fluid therapy and source control for the following case: {vignette_str}"
resuscitation_recommendations = query_engine_guidelines.query(resuscitation_query)

hemodynamic_query = f"Provide recommendations for hemodynamic support and adjunctive therapy, including vasopressors, oxygen, and ventilation support for the following case: {vignette_str}"
hemodynamic_recommendations = query_engine_guidelines.query(hemodynamic_query)

monitoring_query = f"Provide recommendations for monitoring and follow-up, including frequent reassessment and laboratory and imaging studies for the following case: {vignette_str}"
monitoring_recommendations = query_engine_guidelines.query(monitoring_query)

detailed_guideline_recommendations = f"""
### Sepsis Diagnosis Confirmation
{diagnosis_recommendations}

### Initial Resuscitation and Treatment
{resuscitation_recommendations}

### Hemodynamic Support and Adjunctive Therapy
{hemodynamic_recommendations}

### Monitoring and Follow-Up
{monitoring_recommendations}
"""

#print("Compliance Statement:\n", compliance_statement)
#print("Explanation for Non-compliance:\n", compliance_explanation)
#print("Detailed Guideline-based Recommendations:\n", detailed_guideline_recommendations)

# Format the output in Markdown
markdown_output = f"""
# Sepsis Management Recommendations
{sepsis_recommendations}

# Antibiotic Recommendations
{antibiotic_recommendations}

# Compliance Check
{compliance_statement}
  
{compliance_explanation}

# Detailed Guideline-based Recommendations
{detailed_guideline_recommendations}
"""

# Display the output
display(Markdown(markdown_output))

C:\Users\Victor\anaconda3\envs\Python_3_11_10_MA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sepsis Management Recommendations:
 Consider taking additional time to gather information to confirm the diagnosis of sepsis and the suspicion of a bacterial cause. It may be beneficial to perform a work-up as soon as possible and only administer antibiotics when it could truly benefit the patient. New guidelines for the treatment of patients with sepsis should avoid specific time periods for initiating treatment in the general sepsis population.
Antibiotic Recommendations:
 For pulmonary infections like Community-Acquired Pneumonia (CAP), initial empiric therapy typically involves multidrug therapy with a beta-lactam and a macrolide or monotherapy with a respiratory fluoroquinolone. It is important to assess risk factors for MRSA and Pseudomonas aeruginosa and consider adding coverage for these pathogens if necessary. Additionally, adjusting therapy based on local resistance patterns and considering specific circumstances like community-onset MRSA risk are recommended practices in the


# Sepsis Management Recommendations
Consider taking additional time to gather information to confirm the diagnosis of sepsis and the suspicion of a bacterial cause. It may be beneficial to perform a work-up as soon as possible and only administer antibiotics when it could truly benefit the patient. New guidelines for the treatment of patients with sepsis should avoid specific time periods for initiating treatment in the general sepsis population.

# Antibiotic Recommendations
For pulmonary infections like Community-Acquired Pneumonia (CAP), initial empiric therapy typically involves multidrug therapy with a beta-lactam and a macrolide or monotherapy with a respiratory fluoroquinolone. It is important to assess risk factors for MRSA and Pseudomonas aeruginosa and consider adding coverage for these pathogens if necessary. Additionally, adjusting therapy based on local resistance patterns and considering specific circumstances like community-onset MRSA risk are recommended practices in the management of pulmonary infections.

# Compliance Check
Recommendations do not comply with current sepsis management guidelines.
  
The recommendations provided in the literature and antibiotic guidelines emphasize the importance of a thorough diagnostic work-up before initiating antibiotic therapy, tailoring treatment based on individual patient factors, and considering local resistance patterns. These recommendations prioritize the judicious use of antibiotics to ensure optimal patient outcomes and minimize the development of antibiotic resistance. The focus is on personalized treatment strategies rather than rigid time-based approaches, promoting a more patient-centered and evidence-based approach to managing sepsis and pulmonary infections.

To achieve compliance, consider the following suggestions:
To align with the suggestions provided, the new guidelines for sepsis treatment should emphasize the importance of confirming the diagnosis of sepsis and bacterial cause through a thorough work-up before initiating antibiotic therapy. It is recommended to administer antibiotics only when there is a clear benefit to the patient. Avoid setting specific timeframes for initiating treatment in the general sepsis population to allow for individualized care based on clinical assessment and diagnostic findings.

For pulmonary infections such as Community-Acquired Pneumonia (CAP), initial empiric therapy typically involves multidrug therapy with a beta-lactam and a macrolide or monotherapy with a respiratory fluoroquinolone. Risk factors for MRSA and Pseudomonas aeruginosa should be assessed, and coverage for these pathogens should be considered if deemed necessary. It is advisable to adjust therapy based on local resistance patterns and take into account specific circumstances like community-onset MRSA risk when managing pulmonary infections.

# Detailed Guideline-based Recommendations

### Sepsis Diagnosis Confirmation
Avoid using qSOFA as a single screening tool for sepsis in this case of suspected sepsis due to pneumonia in an 86-year-old female with chronic obstructive pulmonary disease, atrial fibrillation, type 2 diabetes mellitus, and hypothyroidism. Instead, consider measuring blood lactate levels as a recommendation for sepsis diagnosis confirmation.

### Initial Resuscitation and Treatment
For initial resuscitation and treatment of the suspected sepsis due to pneumonia in an 86-year-old female with vital signs showing a heart rate of 76 bpm, blood pressure of 103/49 mmHg, respiratory rate of 22 rpm, blood oxygen saturation of 90%, and temperature of 37.2 degrees Celsius, the following recommendations can be made:

1. Begin immediate treatment and resuscitation for sepsis and septic shock.
2. Administer at least 30 mL/kg of IV crystalloid fluid within the first 3 hours of resuscitation for sepsis-induced hypoperfusion or septic shock.
3. Use dynamic measures to guide fluid resuscitation over physical examination or static parameters alone.
4. Guide resuscitation to decrease serum lactate in patients with elevated lactate levels.
5. Use capillary refill time as an adjunct to other measures of perfusion to guide resuscitation in patients with septic shock.

These recommendations aim to address the immediate needs of the patient and optimize fluid resuscitation based on dynamic measures and serum lactate levels to improve outcomes in sepsis and septic shock cases.

### Hemodynamic Support and Adjunctive Therapy
For an 86-year-old female with suspected sepsis due to pneumonia presenting with a heart rate of 76 bpm, blood pressure of 103/49 mmHg, respiratory rate of 22 rpm, blood oxygen saturation of 90%, and a temperature of 37.2 degrees Celsius, the following recommendations can be considered based on the provided context:

1. Hemodynamic Support:
- Consider starting vasopressors peripherally to restore mean arterial pressure promptly.
- Invasive monitoring of arterial blood pressure is recommended over non-invasive monitoring if resources are available.

2. Adjunctive Therapy:
- For septic shock with inadequate mean arterial pressure levels despite norepinephrine and vasopressin, consider adding epinephrine.
- Avoid using terlipressin in adults with septic shock.
- In cases of cardiac dysfunction with persistent hypoperfusion despite adequate volume status and arterial blood pressure, consider adding dobutamine to norepinephrine or using epinephrine alone.
- Avoid using levosimendan for cardiac dysfunction with persistent hypoperfusion in septic shock patients.

3. Oxygen Support:
- For sepsis-induced hypoxemic respiratory failure, consider using high flow nasal oxygen over noninvasive ventilation.

These recommendations are based on the provided context information and guidelines for managing sepsis-related hemodynamic support, adjunctive therapy, and oxygen support.

### Monitoring and Follow-Up
Recommendations for monitoring and follow-up for the case of suspected sepsis due to pneumonia in an 86-year-old female include:
- Frequent reassessment of vital signs including heart rate, blood pressure, respiratory rate, blood oxygen saturation, and temperature.
- Continuous monitoring of laboratory findings such as white blood cell count, blood glucose levels, and blood urea nitrogen.
- Consideration of additional laboratory studies such as blood cultures, procalcitonin levels, and inflammatory markers.
- Imaging studies like chest X-ray or CT scan to assess the extent of pulmonary infection and monitor for any complications.
- Close observation for any signs of clinical deterioration or septic shock, with prompt intervention if necessary.
- Collaboration with infectious disease specialists for guidance on antimicrobial therapy and management of sepsis in the elderly population with comorbidities.

